In [1]:
import ee
import geemap
from ipywidgets import HBox, Label, interact, fixed, VBox
from ipyleaflet import basemaps
from IPython.core.display import HTML 
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive
ee.Initialize()

In [2]:
# Functions

def read_field(*args):
    id_values_ee = aoi_ee.distinct(column_name.value).aggregate_array(column_name.value).getInfo()
    field_name.options = id_values_ee
    
def on_button_clicked(b):
    with output:
        button.disabled=True
        aoi.disabled=True
        start(aoi.value)
        
def add_layer_to_map(column, field):
    selected_feature = aoi_ee.filterMetadata(column, 'equals', float(field)).geometry()
    mapa.clear_layers()
    mapa.add_basemap('HYBRID')
    mapa.addLayer(selected_feature, {'color': 'FF0000'})
    mapa.centerObject(selected_feature, zoom=6)
    
def run_single(column, field, aoi, date): 
    day = [date.day]
    month = [date.month]
    year = [date.year]
    dates_or_series = day, month, year
    run_everything(column, field, aoi, dates_or_series)

def run_range(column, field, aoi, ini_date, end_date):

    days = [ini_date.day, end_date.day]
    months = [ini_date.month, end_date.month]
    years = [ini_date.year, end_date.year]
    dates_or_series = days, months, years
    run_everything(column, field, aoi, dates_or_series)

def run_all_time(column, field, aoi):
    run_everything(column, field, aoi)

def run_everything(column, field, aoi, dates_or_series=False):
    study_area = aoi_ee.filterMetadata(column, 'equals', float(field)).geometry().bounds().coordinates()
    coords = study_area.get(0).getInfo()
    
    ll, ur = coords[0], coords[2]
    minlon, minlat, maxlon, maxlat = ll[0], ll[1], ur[0], ur[1]

    days, months, years = False, False, False
    
    if dates_or_series is not False:
        days, months, years = dates_or_series
    
    attribute = [aoi, field]
    
    print(f'Using the feature: {field} and the aoi: "{aoi}"')
    print('Processing soil moisture maps. This takes a long time')

    %run ~/sepal_pysmm/scripts/run_pysmm.py "$years" "$months" "$days" $minlon $minlat $maxlon $maxlat "$attribute"
    print('done!')

def start(aoi):
    global mapa
    mapa = geemap.Map()
    mapa.clear_layers()
    mapa.clear_controls()
    mapa.add_basemap('HYBRID')
    mapa.layout.height='200px'
    display(mapa)
    
    global aoi_ee
    aoi_ee = ee.FeatureCollection(aoi)
    col_ee = ee.Feature(aoi_ee.first()).propertyNames().getInfo()
    column_name.options = col_ee
    column_name.observe(read_field)

    _ = interactive(add_layer_to_map, column=column_name, field=field_name)
    @interact
    def select_dates(date=dates):
        
        date = dates.value
        kwargs = {
            'column':column_name, 
            'field':field_name, 
            'aoi':fixed(aoi)
        }
        if date == 'Single date':
            kwargs['date'] = unique_date
            w = interactive(run_single, {'manual':True}, **kwargs)
            w.children[3].description = 'Run single date'
            display(w)
            
        elif date == 'Range':
            kwargs['ini_date'] = ini_date
            kwargs['end_date'] = end_date
            w = interactive(run_range, {'manual':True}, **kwargs)
            w.children[4].description = 'Run range'
            display(w)
            
        else:
            w = interactive(run_all_time, {'manual':True}, **kwargs)
            w.children[2].description = 'Run entire series'
            display(w)




In [3]:
# Widgets

dates = widgets.ToggleButtons(
    options=['Single date', 'Range', 'All time series'],
    description='Dates:',
    disabled=False
)

unique_date = widgets.DatePicker(
    description='Date',
    disabled=False
)

ini_date = widgets.DatePicker(
    description='Start date',
    disabled=False
)

end_date = widgets.DatePicker(
    description='End date',
    disabled=False
)

button = widgets.Button(
    description="Use this asset!"
)

output = widgets.Output()

aoi =  widgets.Text(
    value='users/yelenafinegold/107_phu_merged',
    placeholder='Enter your asset ID here',
)

column_name = widgets.Dropdown(description='Column: ')
field_name = widgets.Dropdown(description='Field: ')

In [4]:
# Run
ui1 = HBox([Label('Google Earth Engine asset ID: '), aoi])
button.on_click(on_button_clicked)
display(HBox([ui1, button]))
display(output)

Output()